# Notebook for parser development

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Loading-data" data-toc-modified-id="Loading-data-1">Loading data</a></span></li><li><span><a href="#Check,-remove-duplicates" data-toc-modified-id="Check,-remove-duplicates-2">Check, remove duplicates</a></span></li><li><span><a href="#Column-level-transforms" data-toc-modified-id="Column-level-transforms-3">Column-level transforms</a></span></li><li><span><a href="#Pre-NodeNorming" data-toc-modified-id="Pre-NodeNorming-4">Pre-NodeNorming</a></span><ul class="toc-item"><li><span><a href="#Exploring:-Genes" data-toc-modified-id="Exploring:-Genes-4.1">Exploring: Genes</a></span><ul class="toc-item"><li><span><a href="#HGNC" data-toc-modified-id="HGNC-4.1.1">HGNC</a></span></li><li><span><a href="#OMIM" data-toc-modified-id="OMIM-4.1.2">OMIM</a></span></li><li><span><a href="#Comparing-HGNC-vs-OMIM" data-toc-modified-id="Comparing-HGNC-vs-OMIM-4.1.3">Comparing HGNC vs OMIM</a></span></li><li><span><a href="#Conclusions" data-toc-modified-id="Conclusions-4.1.4">Conclusions</a></span></li></ul></li><li><span><a href="#Exploring:-Diseases" data-toc-modified-id="Exploring:-Diseases-4.2">Exploring: Diseases</a></span><ul class="toc-item"><li><span><a href="#OMIM/orphanet" data-toc-modified-id="OMIM/orphanet-4.2.1">OMIM/orphanet</a></span></li><li><span><a href="#MONDO" data-toc-modified-id="MONDO-4.2.2">MONDO</a></span></li><li><span><a href="#Comparing-OMIM/orphanet-vs-MONDO" data-toc-modified-id="Comparing-OMIM/orphanet-vs-MONDO-4.2.3">Comparing OMIM/orphanet vs MONDO</a></span></li><li><span><a href="#Checking-MONDO-data" data-toc-modified-id="Checking-MONDO-data-4.2.4">Checking MONDO data</a></span></li><li><span><a href="#Conclusions" data-toc-modified-id="Conclusions-4.2.5">Conclusions</a></span></li></ul></li></ul></li><li><span><a href="#Stats-on-rows-removed" data-toc-modified-id="Stats-on-rows-removed-5">Stats on rows removed</a></span></li><li><span><a href="#Adding-NodeNorm-data,-removing-rows" data-toc-modified-id="Adding-NodeNorm-data,-removing-rows-6">Adding NodeNorm data, removing rows</a></span></li><li><span><a href="#Generating-documents" data-toc-modified-id="Generating-documents-7">Generating documents</a></span></li><li><span><a href="#Checking-documents" data-toc-modified-id="Checking-documents-8">Checking documents</a></span></li><li><span><a href="#Parser-notes" data-toc-modified-id="Parser-notes-9">Parser notes</a></span></li></ul></div>

In [1]:
## not for parser. for notebook only 

## CX: allows multiple lines of code to print from one code block
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Loading data

__Current approach__ is to load all files into 1 pandas dataframe. Then I can...

1. check the duplicates situation: records found in multiple panel files. I can check whether the same record looks different between files or not (by checking duplicates using all columns vs key columns). -> Raise errors if yes
2. remove duplicates before generating documents
3. Do some tasks column-wise over all the data, rather than while iterating over rows

Notes:
* There are a few existing BioThings parsers that also use `pandas` to load the entire raw data file at once: https://github.com/search?q=repo%3Abiothings%2Fpending.api%20pandas&type=code
* But there are other parsers that use `csv` to load the file **one row at a time** (generator): https://github.com/search?q=repo%3Abiothings%2Fpending.api+csv+reader&type=code

---

If I did the generator approach (load files 1 by 1, 1 row at a time), I'd have to modify how I do things:
1. Don't do the duplicates check. But try to mitigate potential "duplicate" issues: 
   - Sort all delimited strings
   - Use a hash of all column values (when they're all strings) for `_id`. Want rows with all the same values to produce the same hash
2. Either leave to BioThings toolset to remove duplicates, or could save a running set of `_id` hashes to check if row was already encountered -> not create duplicate docs
3. Do the tasks on single rows/chunks (pandas [read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html#pandas.read_csv) has an iterator for rows/chunks! see iterator/chunksize parameters)

In [2]:
## put into parser: DONE
import pathlib
import pandas as pd

## don't put in parser. Just for this notebook
import glob
from pprint import pprint

## unsure on putting into parser: more for notebook viewing/debugging...
pd.options.display.max_columns = None

<div class="alert alert-block alert-danger">

Adjust the code block below for path/pattern for data files. 
    
This notebook was originally written using data files from the 2025-02-28 release to FTP site 

In [3]:
## put into parser (format): DONE

base_file_path = pathlib.Path.home().joinpath("Desktop", "EBIgene2pheno_files", 
                                              "From_FTP", "2025-02-28")

## uses pathlib's Path.glob, which produces a generator. 
## cast into list so parser code can check if paths were actually matched or not
all_file_paths = list(base_file_path.glob("*.csv.gz"))
all_file_paths

[PosixPath('/Users/colleenxu/Desktop/EBIgene2pheno_files/From_FTP/2025-02-28/CardiacG2P_2025-02-28.csv.gz'),
 PosixPath('/Users/colleenxu/Desktop/EBIgene2pheno_files/From_FTP/2025-02-28/SkeletalG2P_2025-02-28.csv.gz'),
 PosixPath('/Users/colleenxu/Desktop/EBIgene2pheno_files/From_FTP/2025-02-28/DDG2P_2025-02-28.csv.gz'),
 PosixPath('/Users/colleenxu/Desktop/EBIgene2pheno_files/From_FTP/2025-02-28/SkinG2P_2025-02-28.csv.gz'),
 PosixPath('/Users/colleenxu/Desktop/EBIgene2pheno_files/From_FTP/2025-02-28/Hearing_lossG2P_2025-02-28.csv.gz'),
 PosixPath('/Users/colleenxu/Desktop/EBIgene2pheno_files/From_FTP/2025-02-28/CancerG2P_2025-02-28.csv.gz'),
 PosixPath('/Users/colleenxu/Desktop/EBIgene2pheno_files/From_FTP/2025-02-28/EyeG2P_2025-02-28.csv.gz')]

In [4]:
## an example: pathlib's Path.glob produces a generator
## vs glob.glob produces an array (from cwd only?)
base_file_path.glob("*2025-02-28.csv.gz")
glob.glob("*2025-02-28.csv.gz")

<generator object Path.glob at 0x105bf68a0>

[]

In [5]:
## put into parser (format): DONE

## using generator expression (think list/dict comprehension) within pd.concat to load files 1 at a time
## ingesting all columns as str for now
df = pd.concat((pd.read_csv(f, dtype=str) for f in all_file_paths), ignore_index=True)

## make column names snake-case - usable with itertuples later
df.columns = df.columns.str.replace(" ", "_")

In [6]:
df["date_of_last_review"].info(memory_usage="deep")

<class 'pandas.core.series.Series'>
RangeIndex: 4714 entries, 0 to 4713
Series name: date_of_last_review
Non-Null Count  Dtype 
--------------  ----- 
4714 non-null   object
dtypes: object(1)
memory usage: 377.6 KB


In [7]:
## change this column to datetime, saves memory
df["date_of_last_review"] = pd.to_datetime(df["date_of_last_review"])
df["date_of_last_review"].info(memory_usage="deep")

<class 'pandas.core.series.Series'>
RangeIndex: 4714 entries, 0 to 4713
Series name: date_of_last_review
Non-Null Count  Dtype              
--------------  -----              
4714 non-null   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1)
memory usage: 37.0 KB


In [8]:
## I couldn't figure out how to import + ingest column as datetime in 1 step 
## this is what I tried that didn't work

## worked with pandas 2.0.3, but didn't work with pandas 2.2.3: ingested as str
# df = pd.concat((pd.read_csv(f, dtype=str, parse_dates=["date of last review"]) 
#                 for f in all_file_paths), ignore_index=True)

## doesn't work
# df = pd.concat((pd.read_csv(f, dtype=str, parse_dates=["date of last review"], 
#                            date_format="%Y-%m-%d %H:%M:%S%:z") 
#                 for f in all_file_paths), ignore_index=True)
## throws an error
# df = pd.concat((pd.read_csv(f, dtype=str, parse_dates=[["date of last review"]], 
#                            date_format="%Y-%m-%d %H:%M:%S%:z") 
#                 for f in all_file_paths), ignore_index=True)
## throws an error
# df = pd.concat((pd.read_csv(f, dtype={"date of last review": pd.datetime64[ns, tz]})
#                 for f in all_file_paths), ignore_index=True)

In [9]:
df.shape
df.head()

(4714, 21)

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review
0,G2P00124,KCNE1,176261,6240,ISK; JLNS2; LQT5; MINK,KCNE1-related Jervell and Lange-Nielsen syndrome,612347,NaN,biallelic_autosomal,potential secondary finding,strong,altered gene product structure,missense_variant; inframe_deletion; stop_gaine...,undetermined,inferred,NaN,HP:0000407; HP:0001657; HP:0000007; HP:0001279,30461122,DD; Cardiac,KCNE1-related JLNS is due to altered gene prod...,2024-04-05 12:05:01+00:00
1,G2P00841,PTPN11,176876,9644,BPTP3; NS1; PTP2C; SH-PTP2; SHP-2; SHP2,PTPN11-related Noonan syndrome with multiple l...,151100,NaN,monoallelic_autosomal,NaN,definitive,altered gene product structure,missense_variant; inframe_deletion; inframe_in...,undetermined,inferred,NaN,HP:0000325; HP:0002996; HP:0000957; HP:0001709...,27484170; 26377839; 25917897; 25884655; 248207...,DD; Skin; Cardiac,Expert review done on 12/01/2022; Noonan syndr...,2025-01-21 14:56:43+00:00
2,G2P03247,DSC2,125645,3036,CDHF2; DSC3,DSC2-related arrhythmogenic right ventricular ...,NaN,MONDO:0012506,monoallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,inframe_deletion; splice_region_variant; misse...,undetermined,inferred,NaN,NaN,31028357; 23911551; 21636032; 33831308; 263105...,Cardiac,Expert review done on 05/01/2022; DSC2-related...,2024-03-20 09:36:09+00:00
3,G2P03248,DSC2,125645,3036,CDHF2; DSC3,DSC2-related arrhythmogenic right ventricular ...,NaN,MONDO:0012506,biallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,inframe_deletion; splice_region_variant; misse...,undetermined,inferred,NaN,NaN,31028357; 23911551; 21636032; 33831308; 263105...,Cardiac,Expert review done on 05/01/2022; DSC2-related...,2024-03-20 09:35:19+00:00
4,G2P03249,DSG2,125671,3049,CDHF5,DSG2-related arrhythmogenic right ventricular ...,NaN,MONDO:0012434,monoallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,inframe_deletion; missense_variant; stop_gaine...,undetermined,inferred,NaN,NaN,21636032; 33831308; 33917638; 34400560; 240707...,Cardiac,Expert review done on 05/01/2022; DSG2-related...,2024-03-20 09:40:18+00:00


In [10]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4714 entries, 0 to 4713
Data columns (total 21 columns):
 #   Column                              Non-Null Count  Dtype              
---  ------                              --------------  -----              
 0   g2p_id                              4714 non-null   object             
 1   gene_symbol                         4714 non-null   object             
 2   gene_mim                            4712 non-null   object             
 3   hgnc_id                             4714 non-null   object             
 4   previous_gene_symbols               4241 non-null   object             
 5   disease_name                        4714 non-null   object             
 6   disease_mim                         3574 non-null   object             
 7   disease_MONDO                       638 non-null    object             
 8   allelic_requirement                 4714 non-null   object             
 9   cross_cutting_modifier              629 n

## Check, remove duplicates

There are duplicate rows in this dataframe because the record (gene + disease + more) is in several panels (disease falls into multiple categories). This was explored in the data-playground notebook. 

We want to drop those duplicates. 
However, I was concerned that the delimited-string values could differ (only in list order) for the same record in diff files. 
So that's what this check is for. 

In [11]:
## put into parser (format): DONE

n_duplicates_column_combo = df[df.duplicated(subset=["g2p_id"], keep=False)].shape

n_duplicates_all_columns = df[df.duplicated(keep=False)].shape

## for testing
# n_duplicates_all_columns = (1, 1)


if n_duplicates_column_combo != n_duplicates_all_columns: 
    raise AssertionError("The data format has changed, and the assumptions about duplicates/key columns may " \
                          "no longer hold. Re-explore the data and adjust the parser.")

In [12]:
## put into parser (format): DONE

## drop duplicates
df.drop_duplicates(inplace=True, ignore_index=True)

In [13]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3647 entries, 0 to 3646
Data columns (total 21 columns):
 #   Column                              Non-Null Count  Dtype              
---  ------                              --------------  -----              
 0   g2p_id                              3647 non-null   object             
 1   gene_symbol                         3647 non-null   object             
 2   gene_mim                            3645 non-null   object             
 3   hgnc_id                             3647 non-null   object             
 4   previous_gene_symbols               3277 non-null   object             
 5   disease_name                        3647 non-null   object             
 6   disease_mim                         2570 non-null   object             
 7   disease_MONDO                       561 non-null    object             
 8   allelic_requirement                 3647 non-null   object             
 9   cross_cutting_modifier              451 n

## Column-level transforms

Based on data-playground "Notes on parsing data to create documents" section

In [14]:
## double-checking how to add prefixes to OMIM vs orphanet IDs

df_diseasemim = df.copy()

## done to preserve NA
df_diseasemim["disease_mim"] = [i if pd.isna(i) \
                                else "OMIM:" + i if i.isnumeric() \
                                else i \
                                for i in df_diseasemim["disease_mim"]]

df_diseasemim["disease_mim"] = df_diseasemim["disease_mim"].str.replace("Orphanet", "orphanet")

In [15]:
df_diseasemim[df_diseasemim["disease_mim"].str.contains("OMIM:", na=False)].shape

df_diseasemim[df_diseasemim["disease_mim"].str.contains("orphanet:", na=False)].shape

## add up row count. If == num non-null in info above, you're good 
## right now 2570 == 2570, so good

(2561, 21)

(9, 21)

In [16]:
## put into parser (format): DONE

## COLUMN-LEVEL TRANSFORMS

## adding Translator/biolink prefixes to IDs
df["gene_mim"] = "OMIM:" + df["gene_mim"]
df["hgnc_id"] = "HGNC:" + df["hgnc_id"]
df["disease_mim"] = df["disease_mim"].str.replace("Orphanet", "orphanet")
## done to preserve NA
df["disease_mim"] = [i if pd.isna(i)
                     else "OMIM:" + i if i.isnumeric()
                     else i
                     for i in df["disease_mim"]]

## strip whitespace
df["disease_name"] = df["disease_name"].str.strip()
df["comments"] = df["comments"].str.strip()

## create new columns
## UI really wants resource website urls like this. May need to adjust over time as website changes
df["g2p_record_url"] = "https://www.ebi.ac.uk/gene2phenotype/lgd/" +  df["g2p_id"]

## replace panel keywords with full names shown on G2P website for single record
## keeping "Hearing loss" as-is, changing all other values
df["panel"] = df["panel"].str.replace("DD", "Developmental disorders")
df["panel"] = df["panel"].str.replace("Cancer", "Cancer disorders")
df["panel"] = df["panel"].str.replace("Cardiac", "Cardiac disorders")
df["panel"] = df["panel"].str.replace("Eye", "Eye disorders")
df["panel"] = df["panel"].str.replace("Skeletal", "Skeletal disorders")
df["panel"] = df["panel"].str.replace("Skin", "Skin disorders")

In [17]:
## checking on column-level transforms

df.head()
# df["g2p record url"].unique()[0:100]

# df[df["disease mim"].str.contains("orphanet", na=False)]  ## 9 rows, so that's correct
# df[df["panel"].str.contains("Hearing", na=False)]

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,g2p_record_url
0,G2P00124,KCNE1,OMIM:176261,HGNC:6240,ISK; JLNS2; LQT5; MINK,KCNE1-related Jervell and Lange-Nielsen syndrome,OMIM:612347,NaN,biallelic_autosomal,potential secondary finding,strong,altered gene product structure,missense_variant; inframe_deletion; stop_gaine...,undetermined,inferred,NaN,HP:0000407; HP:0001657; HP:0000007; HP:0001279,30461122,Developmental disorders; Cardiac disorders,KCNE1-related JLNS is due to altered gene prod...,2024-04-05 12:05:01+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00124
1,G2P00841,PTPN11,OMIM:176876,HGNC:9644,BPTP3; NS1; PTP2C; SH-PTP2; SHP-2; SHP2,PTPN11-related Noonan syndrome with multiple l...,OMIM:151100,NaN,monoallelic_autosomal,NaN,definitive,altered gene product structure,missense_variant; inframe_deletion; inframe_in...,undetermined,inferred,NaN,HP:0000325; HP:0002996; HP:0000957; HP:0001709...,27484170; 26377839; 25917897; 25884655; 248207...,Developmental disorders; Skin disorders; Cardi...,Expert review done on 12/01/2022; Noonan syndr...,2025-01-21 14:56:43+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00841
2,G2P03247,DSC2,OMIM:125645,HGNC:3036,CDHF2; DSC3,DSC2-related arrhythmogenic right ventricular ...,NaN,MONDO:0012506,monoallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,inframe_deletion; splice_region_variant; misse...,undetermined,inferred,NaN,NaN,31028357; 23911551; 21636032; 33831308; 263105...,Cardiac disorders,Expert review done on 05/01/2022; DSC2-related...,2024-03-20 09:36:09+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03247
3,G2P03248,DSC2,OMIM:125645,HGNC:3036,CDHF2; DSC3,DSC2-related arrhythmogenic right ventricular ...,NaN,MONDO:0012506,biallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,inframe_deletion; splice_region_variant; misse...,undetermined,inferred,NaN,NaN,31028357; 23911551; 21636032; 33831308; 263105...,Cardiac disorders,Expert review done on 05/01/2022; DSC2-related...,2024-03-20 09:35:19+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03248
4,G2P03249,DSG2,OMIM:125671,HGNC:3049,CDHF5,DSG2-related arrhythmogenic right ventricular ...,NaN,MONDO:0012434,monoallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,inframe_deletion; missense_variant; stop_gaine...,undetermined,inferred,NaN,NaN,21636032; 33831308; 33917638; 34400560; 240707...,Cardiac disorders,Expert review done on 05/01/2022; DSG2-related...,2024-03-20 09:40:18+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03249


## Pre-NodeNorming

Querying NodeNorm: send unique values (no duplicates!) from entire column in large batches -> generate mapping dict to use. 
<br>
__Not querying 1-by-1 or 1 row at a time: much slower__ and would involve sending duplicate IDs (unless saved dict is kept outside loop and checked) 

Not going to use NameResolver: not optimistic this would work anyways. My manual process of getting "better" disease IDs is to use the gene IDs, find the diseases they're linked to in OMIM and Monarch, and seeing if those match the data's disease name / phenotypes / publications. This is more complicated than just using NameResolver.

<div class="alert alert-block alert-danger">

Set the NodeNorm URL you want to use. 

In [18]:
## put into parser (format): DONE

import requests

## from BioThings annotator code: for interoperability between diff Python versions
# try:
#     from itertools import batched  # new in Python 3.12
# except ImportError:
#     from itertools import islice

#     def batched(iterable, n):
#         # batched('ABCDEFG', 3) → ABC DEF G
#         if n < 1:
#             raise ValueError("n must be at least one")
#         iterator = iter(iterable)
#         while batch := tuple(islice(iterator, n)):
#             yield batch

## doing to test that this works
from itertools import islice

def batched(iterable, n):
    # batched('ABCDEFG', 3) → ABC DEF G
    if n < 1:
        raise ValueError("n must be at least one")
    iterator = iter(iterable)
    while batch := tuple(islice(iterator, n)):
        yield batch

nodenorm_url = "https://nodenorm.ci.transltr.io/get_normalized_nodes"

### Exploring: Genes

Every row has at least 1 gene ID (HGNC column has no missing values). So no rows will be removed because there's no gene IDs to use for the pre-NodeNorming. 

In [19]:
df[["gene_symbol", "hgnc_id", "gene_mim"]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3647 entries, 0 to 3646
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   gene_symbol  3647 non-null   object
 1   hgnc_id      3647 non-null   object
 2   gene_mim     3645 non-null   object
dtypes: object(3)
memory usage: 85.6+ KB


In [20]:
df[df["gene_mim"].isna()]

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,g2p_record_url
1208,G2P00977,ZNF599,NaN,HGNC:26408,FLJ30663,ZNF599-related NOT IN OMIM,NaN,NaN,monoallelic_autosomal,NaN,limited,uncertain,NaN,undetermined,inferred,NaN,NaN,NaN,Developmental disorders,NaN,2015-07-22 16:15:03+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00977
3440,G2P02168,MFSD6L,NaN,HGNC:26656,FLJ35773,MFSD6L-related congenital cataract,NaN,MONDO:0005129,biallelic_autosomal,NaN,limited,altered gene product structure,missense_variant; inframe_deletion; inframe_in...,undetermined,inferred,NaN,HP:0010864; HP:0000007,22935719,Eye disorders,NaN,2017-08-29 09:35:13+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P02168


In [21]:
## saving stats on data with no gene IDs, just in case

stats_no_gene_IDs = {
    "n_rows": df[df["gene_mim"].isna() & df["hgnc_id"].isna()].shape[0],
    "n_names": len(df[df["gene_mim"].isna() & df["hgnc_id"].isna()]["gene_symbol"].unique())
}

stats_no_gene_IDs["n_rows"]
stats_no_gene_IDs["n_names"]

0

0

#### HGNC

__Running Gene HGNC IDs through NodeNorm__


Catching potential mapping failures for later stats report

In [22]:
## saving stats on data with no HGNC IDs, just in case

n_rows_no_hgnc = df["hgnc_id"].isna().sum()

In [23]:
## get set of unique CURIEs to put into NodeNorm
hgnc = df["hgnc_id"].dropna().unique()
len(hgnc)

2991

In [24]:
hgnc_nodenorm_mapping = {}

## set up variables to catch potential mapping failures
stats_hgnc_mapping_failures = {
    "unexpected_error": {},
    "nodenorm_returned_none": [],
    "wrong_category": {},
    "no_label": []
    
}

In [25]:
## larger batches are quicker
for batch in batched(hgnc, 1000):
    ## returns tuples -> cast to list
    req_body = {
        "curies": list(batch),
        "conflate": True,
    }
    r = requests.post(nodenorm_url, json=req_body)
    response = r.json()
    
    ## not doing dict comprehension. allows easier review, logic writing
    for k,v in response.items():
        ## catch unexpected errors
        try:
            ## if NodeNorm didn't have info on this ID, v will be None
            if v is not None:
                ## don't keep mapping if category is not the expected one
                if v["type"][0] == "biolink:Gene":
                    ## also throw out mapping if no primary label found
                    if v["id"].get("label"):
                        temp = {
                            k: {"primary_id": v["id"]["identifier"],
                                "primary_label": v["id"]["label"]
                               }
                        }
                        hgnc_nodenorm_mapping.update(temp)
                    else:
                        stats_hgnc_mapping_failures["no_label"].append(k)
#                         print(f"{k}: NodeNorm didn't find primary label. Not keeping this mapping.")
                else:
                    stats_hgnc_mapping_failures["wrong_category"].update({k: v["type"][0]})
#                     print(f'{k}: NodeNorm found different category {v["type"][0]}. Not keeping this mapping.')
            else:
                stats_hgnc_mapping_failures["nodenorm_returned_none"].append(k)
#                 print(f"{k}: NodeNorm didn't recognize this ID")
        except:
            stats_hgnc_mapping_failures["unexpected_error"].update({k: v})
            print(f'Encountered an unexpected error.')
            print(f'NodeNorm response key: {k}')
            print(f'NodeNorm response value: {v}')

In [26]:
len(hgnc_nodenorm_mapping)

stats_hgnc_mapping_failures

2991

{'unexpected_error': {},
 'nodenorm_returned_none': [],
 'wrong_category': {},
 'no_label': []}

#### OMIM

__Running Gene OMIM IDs through NodeNorm__

Catching potential mapping failures for later stats report. 

Pasted, adjusted from HGNC code blocks above.

In [27]:
## get set of unique CURIEs to put into NodeNorm
gene_omim = df["gene_mim"].dropna().unique()
len(gene_omim)

2989

In [28]:
gene_omim_nodenorm_mapping = {}

## set up variables to catch potential mapping failures
stats_gene_omim_mapping_failures = {
    "unexpected_error": {},
    "nodenorm_returned_none": [],
    "wrong_category": {},
    "no_label": []
    
}

In [29]:
## larger batches are quicker
for batch in batched(gene_omim, 1000):
    ## returns tuples -> cast to list
    req_body = {
        "curies": list(batch),
        "conflate": True,
    }
    r = requests.post(nodenorm_url, json=req_body)
    response = r.json()
    
    ## not doing dict comprehension. allows easier review, logic writing
    for k,v in response.items():
        ## catch unexpected errors
        try:
            ## if NodeNorm didn't have info on this ID, v will be None
            if v is not None:
                ## don't keep mapping if category is not the expected one
                if v["type"][0] == "biolink:Gene":
                    ## also throw out mapping if no primary label found
                    if v["id"].get("label"):
                        temp = {
                            k: {"primary_id": v["id"]["identifier"],
                                "primary_label": v["id"]["label"]
                               }
                        }
                        gene_omim_nodenorm_mapping.update(temp)
                    else:
                        stats_gene_omim_mapping_failures["no_label"].append(k)
#                         print(f"{k}: NodeNorm didn't find primary label. Not keeping this mapping.")
                else:
                    stats_gene_omim_mapping_failures["wrong_category"].update({k: v["type"][0]})
#                     print(f'{k}: NodeNorm found different category {v["type"][0]}. Not keeping this mapping.')
            else:
                stats_gene_omim_mapping_failures["nodenorm_returned_none"].append(k)
#                 print(f"{k}: NodeNorm didn't recognize this ID")
        except:
            stats_gene_omim_mapping_failures["unexpected_error"].update({k: v})
            print(f'Encountered an unexpected error.')
            print(f'NodeNorm response key: {k}')
            print(f'NodeNorm response value: {v}')

In [30]:
len(gene_omim_nodenorm_mapping)

stats_gene_omim_mapping_failures

2989

{'unexpected_error': {},
 'nodenorm_returned_none': [],
 'wrong_category': {},
 'no_label': []}

In [31]:
## from looking at 2025-03-28 data
# response["OMIM:621003"]

#### Comparing HGNC vs OMIM

In [32]:
## if row has both IDs, look for diff in mappings from each ID
for row in df[["gene_mim", "hgnc_id"]].itertuples(index=False):
    ## has both IDs
    if pd.notna(row.gene_mim) and pd.notna(row.hgnc_id):
        ## if have NodeNorm mappings for both
        if gene_omim_nodenorm_mapping.get(row.gene_mim) and \
        hgnc_nodenorm_mapping.get(row.hgnc_id):
            ## check if mappings are diff
            if gene_omim_nodenorm_mapping[row.gene_mim]["primary_id"] != \
            hgnc_nodenorm_mapping[row.hgnc_id]["primary_id"]:
                print(row)

## nothing prints, so there are no mismatches

In [33]:
## look for differences in name between NodeNormed and original data

for row in df[["gene_symbol", "hgnc_id"]].itertuples(index=False):
    ## works because both columns have no missing values and there's no failed mappings
    ## if this changes, need to adjust this code block
    if row.gene_symbol != hgnc_nodenorm_mapping[row.hgnc_id]["primary_label"]:
        print(f"G2P name {row.gene_symbol}, ID {row.hgnc_id}")
        print(f'NodeNorm name {hgnc_nodenorm_mapping[row.hgnc_id]["primary_label"]}, ID {hgnc_nodenorm_mapping[row.hgnc_id]["primary_id"]}')
        print("\n")

G2P name MT-TP, ID HGNC:7494
NodeNorm name TRNP, ID NCBIGene:4571


G2P name CENPJ, ID HGNC:17272
NodeNorm name CPAP, ID NCBIGene:55835


G2P name CCDC103, ID HGNC:32700
NodeNorm name DNAAF19, ID NCBIGene:388389


G2P name MT-TL1, ID HGNC:7490
NodeNorm name TRNL1, ID NCBIGene:4567


G2P name MT-ND1, ID HGNC:7455
NodeNorm name ND1, ID NCBIGene:4535


G2P name MT-ND4, ID HGNC:7459
NodeNorm name ND4, ID NCBIGene:4538


G2P name MT-ATP6, ID HGNC:7414
NodeNorm name ATP6, ID NCBIGene:4508


G2P name MT-ND5, ID HGNC:7461
NodeNorm name ND5, ID NCBIGene:4540


G2P name MT-ND6, ID HGNC:7462
NodeNorm name ND6, ID NCBIGene:4541




Review of mismatched names:
* NodeNorm is correct that CENPJ should be CPAP, CCDC103 -> DNAAF19
* The rest look like mitochondrial genes, and NCBIGene main name seems to match G2P name, not NodeNorm -> messaged NodeNorm

#### Conclusions

<div class="alert alert-block alert-success">

__Exploration__

* no mapping failures
* when rows have both OMIM and HGNC IDs, there were no differences in NodeNorm mapping ("mismatches")
    
__Decision: Use HGNC ID column to generate NodeNorm values__

* less missing values (none right now)
* these IDs are probably only genes (vs OMIM terms can be multiple types) 

### Exploring: Diseases

There are many more missing IDs for Disease, compared to Gene. 

As mentioned at the beginning of the "Pre-NodeNorming" section, I won't be using NameResolver right now. 

__This means all rows w/o any disease IDs will be removed__ because they aren't pre-NodeNormed. 

In [34]:
df[["disease_name", "disease_mim", "disease_MONDO"]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3647 entries, 0 to 3646
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   disease_name   3647 non-null   object
 1   disease_mim    2570 non-null   object
 2   disease_MONDO  561 non-null    object
dtypes: object(3)
memory usage: 85.6+ KB


In [35]:
df[df["disease_mim"].isna() & df["disease_MONDO"].isna()]

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,g2p_record_url
67,G2P03443,CRYAB,OMIM:123590,HGNC:2389,CRYA2; HSPB5,CRYAB-related alpha-related B crystallinopathy,NaN,NaN,monoallelic_autosomal,NaN,definitive,altered gene product structure,missense_variant; stop_gained_NMD_escaping; fr...,dominant negative,inferred,NaN,HP:0000518; HP:0003555; HP:0009027; HP:0003236...,11577372; 23590293; 30681346; 570292; 14681890...,Developmental disorders; Eye disorders; Cardia...,CRYAB-related alpha-crystallinopathy is typica...,2025-01-21 12:59:51+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03443
69,G2P03445,GAA,OMIM:606800,HGNC:4065,NaN,GAA-related Pompe disease,NaN,NaN,biallelic_autosomal,restricted mutation set,definitive,decreased gene product level; altered gene pro...,NaN,undetermined,inferred,NaN,NaN,30681346; 31254424; 1652892; 8094613; 7981676;...,Cardiac disorders,Pompe disease is inherited as an autosomal rec...,2024-03-26 10:53:54+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03445
82,G2P00021,COL1A1,OMIM:120150,HGNC:2197,OI4,COL1A1-related osteogenesis imperfecta spectrum,NaN,NaN,monoallelic_autosomal,restricted mutation set,definitive,altered gene product structure,NaN,dominant negative,inferred,NaN,HP:0003502; HP:0000707; HP:0000325; HP:0000923...,9295084; 3082886; 18409203; 2295701; 1988452; ...,Developmental disorders; Skin disorders; Skele...,NaN,2025-01-15 11:51:09+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00021
218,G2P00872,PIK3CA,OMIM:171834,HGNC:8975,PI3K,PIK3CA-related overgrowth spectrum disorder wi...,NaN,NaN,monoallelic_autosomal,typically mosaic; restricted mutation set,definitive,altered gene product structure,NaN,gain of function,inferred,NaN,HP:0008678; HP:0010301; HP:0009748; HP:0001355...,22658544; 22729224,Developmental disorders; Skeletal disorders,NaN,2024-12-11 11:40:22+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00872
399,G2P02554,PRRX1,OMIM:167420,HGNC:9142,PHOX1; PMX1,PRRX1-related craniosynostosis,NaN,NaN,monoallelic_autosomal,NaN,moderate,decreased gene product level; altered gene pro...,frameshift_variant; missense_variant; stop_gained,undetermined,inferred,NaN,HP:0001363,37154149,Developmental disorders; Skeletal disorders,NaN,2024-03-22 10:30:40+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P02554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3609,G2P02428,IPO13,OMIM:610411,HGNC:16853,IMP13; KIAA0724; RANBP13,"IPO13-related ocular coloboma, microphthalmia,...",NaN,NaN,biallelic_autosomal,NaN,strong,absent gene product,NaN,loss of function,inferred,NaN,HP:0000482; HP:0000007; HP:0000612; HP:0000568,29700284,Eye disorders,NaN,2018-05-25 14:49:39+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P02428
3612,G2P02432,MAFB,OMIM:608968,HGNC:6408,KRML,MAFB-related focal segmental glomerulosclerosi...,NaN,NaN,monoallelic_autosomal,NaN,strong,altered gene product structure,missense_variant; inframe_deletion; inframe_in...,undetermined,inferred,NaN,NaN,29779709,Eye disorders,NaN,2018-05-30 15:12:32+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P02432
3618,G2P02540,IFT88,OMIM:600595,HGNC:20606,D13S1056E; HTG737; MGC26259; TG737; TTC10,IFT88-related non-syndromic retinal degeneration,NaN,NaN,biallelic_autosomal,NaN,limited,absent gene product,NaN,loss of function,inferred,NaN,HP:0007947; HP:0000548; HP:0000662; HP:0000546...,29978320,Eye disorders,NaN,2025-01-16 11:09:04+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P02540
3634,G2P02823,MFRP,OMIM:606227,HGNC:18121,C1QTNF5; FLJ30570; NNO2; RD6,MFRP-related non-syndromic retinitis pigmenta,NaN,NaN,biallelic_autosomal,NaN,definitive,uncertain,NaN,undetermined,inferred,NaN,HP:0000510; HP:0000556,24474277; 22605

In [36]:
## saving stats on data with no disease IDs

stats_no_disease_IDs = {
    "n_rows": df[df["disease_mim"].isna() & df["disease_MONDO"].isna()].shape[0],
    "n_names": len(df[df["disease_mim"].isna() & df["disease_MONDO"].isna()]["disease_name"].unique())
}

stats_no_disease_IDs["n_rows"]
stats_no_disease_IDs["n_names"]

633

621

#### OMIM/orphanet

__Running OMIM/orphanet IDs through NodeNorm__

Catching mapping failures for later stats report

Pasted, adjusted from HGNC code blocks above.

In [37]:
## put into parser (format): DONE

## get set of unique CURIEs to put into NodeNorm
disease_OmOr = df["disease_mim"].dropna().unique()
len(disease_OmOr)

2401

In [38]:
## put into parser (format): DONE

OmOr_nodenorm_mapping = {}

## set up variables to catch mapping failures
stats_OmOr_mapping_failures = {
    "unexpected_error": {},
    "nodenorm_returned_none": [],
    "wrong_category": {},
    "no_label": []
    
}

In [39]:
## put into parser (format): DONE

## larger batches are quicker
for batch in batched(disease_OmOr, 1000):
    ## returns tuples -> cast to list
    req_body = {
        "curies": list(batch),
        "conflate": True,
    }
    r = requests.post(nodenorm_url, json=req_body)
    response = r.json()
    
    ## not doing dict comprehension. allows easier review, logic writing
    for k,v in response.items():
        ## catch unexpected errors
        try:
            ## if NodeNorm didn't have info on this ID, v will be None
            if v is not None:
                ## don't keep mapping if category is not the expected one
                if v["type"][0] == "biolink:Disease":
                    ## also throw out mapping if no primary label found
                    if v["id"].get("label"):
                        temp = {
                            k: {"primary_id": v["id"]["identifier"],
                                "primary_label": v["id"]["label"]
                               }
                        }
                        OmOr_nodenorm_mapping.update(temp)
                    else:
                        stats_OmOr_mapping_failures["no_label"].append(k)
#                         print(f"{k}: NodeNorm didn't find primary label. Not keeping this mapping.")
                else:
                    stats_OmOr_mapping_failures["wrong_category"].update({k: v["type"][0]})
#                     print(f'{k}: NodeNorm found different category {v["type"][0]}. Not keeping this mapping.')
            else:
                stats_OmOr_mapping_failures["nodenorm_returned_none"].append(k)
#                 print(f"{k}: NodeNorm didn't recognize this ID")
        except:
            stats_OmOr_mapping_failures["unexpected_error"].update({k: v})
            print(f'Encountered an unexpected error.')
            print(f'NodeNorm response key: {k}')
            print(f'NodeNorm response value: {v}')

In [40]:
## put into parser (format): DONE

## calculate stats: number of rows affected by each type of mapping failure
n_rows_no_data = df[
    df["disease_mim"].isin(stats_OmOr_mapping_failures["nodenorm_returned_none"])
].shape[0]
n_rows_wrong_category = df[
    df["disease_mim"].isin(stats_OmOr_mapping_failures["wrong_category"].keys())
].shape[0]
n_rows_no_label = df[
    df["disease_mim"].isin(stats_OmOr_mapping_failures["no_label"])
].shape[0]

In [41]:
len(OmOr_nodenorm_mapping)

stats_OmOr_mapping_failures["unexpected_error"]

len(stats_OmOr_mapping_failures["nodenorm_returned_none"])
len(stats_OmOr_mapping_failures["wrong_category"])
len(stats_OmOr_mapping_failures["no_label"])

2333

{}

39

26

3

In [42]:
## code used to review mapping failures 

# stats_OmOr_mapping_failures["nodenorm_returned_none"][5:10]

# stats_OmOr_mapping_failures["wrong_category"]

# stats_OmOr_mapping_failures["no_label"]

In [43]:
## code used to review mapping failures 

# df[df["disease_mim"] == "OMIM:613180"]

<div class="alert alert-block alert-info">

__Reviewed Disease OMIM/orphanet NodeNorm mapping failures__

All were OMIM IDs, none were orphanet. 
    
39 cases where NodeNorm returned None (didn't recognize/resolve ID). __I checked some (10)__:
* 5: ID has been replaced/moved to a diff ID (OMIM:607236, OMIM:608890, OMIM:613180, OMIM:300706, OMIM:300141) -> __TELL EBI GENE2PHENO__
* 3: ID doesn't exist (OMIM:249163, OMIM:319029, OMIM:237145) -> __TELL EBI GENE2PHENO__
* 1: NodeNorm error - this is a valid disease ID that it should recognize (OMIM:133700) -> messaged NodeNorm
* 1: valid ID, but it doesn't seem to be a disease. There may be better IDs out there (OMIM:601884) -> __TELL EBI GENE2PHENO__, messaged NodeNorm

26 cases where NodeNorm category was something else (currently, always Gene). I checked all: 
* 25: NodeNorm is correct, this is a gene -> __TELL EBI GENE2PHENO__
* 1: NodeNorm error - this is a valid disease ID (OMIM:188400) -> messaged NodeNorm
    
3 cases where NodeNorm didn't have a primary label. I checked all:
* 2: NodeNorm error - these are valid disease IDs with labels (OMIM:620987, OMIM:620964) -> messaged NodeNorm
* 1: valid ID, but it doesn't seem to be a disease. EBI gene2pheno shouldn't use, not sure it should be in NodeNorm -> __TELL EBI GENE2PHENO__, messaged NodeNorm
    
</div>

In [44]:
## code used to check for orphanet mapping failures 

for i in stats_OmOr_mapping_failures["nodenorm_returned_none"]:
    if "orphanet" in i:
        print(i)
        
for i in stats_OmOr_mapping_failures["wrong_category"].keys():
    if "orphanet" in i:
        print(i)

for i in stats_OmOr_mapping_failures["no_label"]:
    if "orphanet" in i:
        print(i)

No orphanet IDs were had mapping failures but I checked all (9) mappings anyways - they looked fine. 

In [45]:
df[df["disease_mim"].str.contains("orphanet", na=False)]

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,g2p_record_url
2052,G2P02555,SMAD6,OMIM:602931,HGNC:6772,HST17432; MADH6; MADH7,SMAD6-related non-syndromic craniosynostosis,orphanet:139390,NaN,monoallelic_autosomal,NaN,limited,absent gene product,NaN,loss of function,inferred,NaN,HP:0001363,27606499; 28808027,Developmental disorders,NaN,2019-04-17 12:18:34+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P02555
2061,G2P02564,TONSL,OMIM:604546,HGNC:7801,IKBR; NFKBIL2,TONSL-related sponastrime dysplasia,orphanet:93357,NaN,biallelic_autosomal,NaN,strong,absent gene product,NaN,loss of function,inferred,NaN,HP:0005281; HP:0100255; HP:0002650; HP:0004322,30773277; 30773278,Developmental disorders,NaN,2018-11-07 09:53:40+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P02564
2846,G2P01819,RET,OMIM:164761,HGNC:9967,CDHF12; CDHR16; HSCR1; MEN2A; MEN2B; MTC1; PTC...,RET-related medullary thyroid carcinoma,orphanet:1332,NaN,monoallelic_autosomal,NaN,definitive,uncertain,NaN,undetermined,inferred,NaN,HP:0002865,10323403; 11454140; 14602786; 15240641; 950672...,Skin disorders; Cancer disorders,NaN,2017-09-01 16:19:16+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P01819
2967,G2P02669,INSR,OMIM:147670,HGNC:6091,CD220,INSR-related leprechaunism,orphanet:508,NaN,biallelic_autosomal,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,HP:0000119; HP:0001574; HP:0000138; HP:0000212...,8105179; 7815442,Skin disorders,NaN,2019-09-16 15:47:00+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P02669
3197,G2P03354,SEC23B,OMIM:610512,HGNC:10702,CDA-II; CDAII; CDAN2; HEMPAS,SEC23B-related Cowden syndrome,orphanet:201,NaN,monoallelic_autosomal,NaN,limited,uncertain,NaN,gain of function,inferred,NaN,HP:0012114; HP:0012056; HP:0500009; HP:0005584...,26522472,Cancer disorders,NaN,2022-11-30 08:49:50+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03354
3304,G2P02001,RCBTB1,OMIM:607867,HGNC:18243,CLLD7; CLLL7; FLJ10716,RCBTB1-related familial exudative vitreoretino...,orphanet:891,NaN,monoallelic_autosomal,NaN,limited,altered gene product structure,missense_variant; inframe_deletion; inframe_in...,undetermined,inferred,NaN,HP:0012231; HP:0000006; HP:0011532; HP:0001141...,26908610,Eye disorders,NaN,2017-06-11 18:14:48+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P02001
3484,G2P02222,REV3L,OMIM:602776,HGNC:9968,POLZ; REV3,REV3L-related Moebius syndrome,orphanet:570,NaN,monoallelic_autosomal,NaN,limited,uncertain,NaN,undetermined,inferred,NaN,NaN,26068067,Eye disorders,NaN,2017-08-30 12:00:00+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P02222
3526,G2P02286,TMEM98,OMIM:615949,HGNC:24529,DKFZP564K1964,TMEM98-related nanophthalmos,orphanet:35612,NaN,monoallelic_autosomal,NaN,strong,uncertain,NaN,undetermined,inferred,NaN,NaN,24852644; 26392740,Eye disorders,NaN,2017-08-31 13:26:51+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P02286
3607,G2P02426,TENM3,OMIM:610083,HGNC:29944,KIAA1455; ODZ3; TEN-M3; TEN3,TENM3-related colobomatous microphthalmia,orphanet:98938,NaN,biallelic_autosomal,NaN,strong,absent gene product,NaN,loss of function,inferred,NaN,HP:0000567; HP:0007700; HP:0000565; HP:0000482...,29753094; 27103084; 22766609,Eye disorders,NaN,2018-05-25 09:59:48+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P02426


In [46]:
OmOr_nodenorm_mapping["orphanet:1332"]

{'primary_id': 'MONDO:0015277',
 'primary_label': 'medullary thyroid gland carcinoma'}

<div class="alert alert-block alert-success">

I decided <b>not to try using MONDO mappings when the OMIM mapping failed</b>, because there's only a few cases where those rows even have MONDO IDs to use. 

* nodenorm_returned_none (39): none have MONDO
* wrong_category (26): only 2 have MONDO
* no_label (3): none have MONDO

In [47]:
## code used to check how many rows have OMIM failure + MONDO ID 

df[df["disease_mim"].isin(stats_OmOr_mapping_failures["nodenorm_returned_none"]) & 
   df["disease_MONDO"].notna()].shape

df[df["disease_mim"].isin(stats_OmOr_mapping_failures["wrong_category"].keys()) & 
   df["disease_MONDO"].notna()].shape

df[df["disease_mim"].isin(stats_OmOr_mapping_failures["no_label"]) & 
   df["disease_MONDO"].notna()].shape

(0, 22)

(2, 22)

(0, 22)

#### MONDO

__Running MONDO IDs through NodeNorm__

Catching potential mapping failures for later stats report

Pasted, adjusted from Disease OMIM/orphanet code blocks above.

In [48]:
## get set of unique CURIEs to put into NodeNorm
mondo = df["disease_MONDO"].dropna().unique()
len(mondo)

380

In [49]:
mondo_nodenorm_mapping = {}

## set up variables to catch mapping failures
stats_mondo_mapping_failures = {
    "unexpected_error": {},
    "nodenorm_returned_none": [],
    "wrong_category": {},
    "no_label": []
    
}

In [50]:
## larger batches are quicker
for batch in batched(mondo, 1000):
    ## returns tuples -> cast to list
    req_body = {
        "curies": list(batch),
        "conflate": True,
    }
    r = requests.post(nodenorm_url, json=req_body)
    response = r.json()
    
    ## not doing dict comprehension. allows easier review, logic writing
    for k,v in response.items():
        ## catch unexpected errors
        try:
            ## if NodeNorm didn't have info on this ID, v will be None
            if v is not None:
                ## don't keep mapping if category is not the expected one
                if v["type"][0] == "biolink:Disease":
                    ## also throw out mapping if no primary label found
                    if v["id"].get("label"):
                        temp = {
                            k: {"primary_id": v["id"]["identifier"],
                                "primary_label": v["id"]["label"]
                               }
                        }
                        mondo_nodenorm_mapping.update(temp)
                    else:
                        stats_mondo_mapping_failures["no_label"].append(k)
#                         print(f"{k}: NodeNorm didn't find primary label. Not keeping this mapping.")
                else:
                    stats_mondo_mapping_failures["wrong_category"].update({k: v["type"][0]})
#                     print(f'{k}: NodeNorm found different category {v["type"][0]}. Not keeping this mapping.')
            else:
                stats_mondo_mapping_failures["nodenorm_returned_none"].append(k)
#                 print(f"{k}: NodeNorm didn't recognize this ID")
        except:
            stats_mondo_mapping_failures["unexpected_error"].update({k: v})
            print(f'Encountered an unexpected error.')
            print(f'NodeNorm response key: {k}')
            print(f'NodeNorm response value: {v}')

In [51]:
len(mondo_nodenorm_mapping)

stats_mondo_mapping_failures

380

{'unexpected_error': {},
 'nodenorm_returned_none': [],
 'wrong_category': {},
 'no_label': []}

#### Comparing OMIM/orphanet vs MONDO

In [52]:
## if row has both IDs, look for diff in mappings from each ID

## list of tuples (omim/orpha, mondo)
mismatches = []

for row in df[["disease_mim", "disease_MONDO"]].itertuples(index=False):
    ## has both IDs
    if pd.notna(row.disease_mim) and pd.notna(row.disease_MONDO):
        ## if have NodeNorm mappings for both
        if OmOr_nodenorm_mapping.get(row.disease_mim) and \
        mondo_nodenorm_mapping.get(row.disease_MONDO):
            ## check if mappings are diff
            if OmOr_nodenorm_mapping[row.disease_mim]["primary_id"] != \
            mondo_nodenorm_mapping[row.disease_MONDO]["primary_id"]:
                mismatches.append((row.disease_mim, row.disease_MONDO))

print(f"There's {len(mismatches)} mismatches between the OMIM/orphanet and MONDO NodeNorm mappings.")

There's 22 mismatches between the OMIM/orphanet and MONDO NodeNorm mappings.


In [53]:
## code chunk to review mismatches 1 by 1
mismatches[21]

('OMIM:175800', 'MONDO:0006602')

In [54]:
## code chunk to review mismatches 1 by 1

OmOr_nodenorm_mapping["OMIM:175800"]
mondo_nodenorm_mapping["MONDO:0006602"]

{'primary_id': 'MONDO:0008290',
 'primary_label': 'porokeratosis 1, Mibelli type'}

{'primary_id': 'MONDO:0006602', 'primary_label': 'porokeratosis'}

In [55]:
## code chunk to review mismatches 1 by 1

df[df["disease_mim"] == "OMIM:175800"]
df[df["disease_MONDO"] == "MONDO:0006602"]

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,g2p_record_url
3084,G2P03623,PMVK,OMIM:607622,HGNC:9141,HUMPMKI; PMK; PMKA,"PMVK-related porokeratosis, multiple types",OMIM:175800,MONDO:0006602,monoallelic_autosomal,NaN,definitive,altered gene product structure,NaN,undetermined,inferred,NaN,HP:0000006; HP:0008069; HP:0001036; HP:0003220...,30942823,Skin disorders,NaN,2025-01-09 15:40:24+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03623


,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,g2p_record_url
3084,G2P03623,PMVK,OMIM:607622,HGNC:9141,HUMPMKI; PMK; PMKA,"PMVK-related porokeratosis, multiple types",OMIM:175800,MONDO:0006602,monoallelic_autosomal,NaN,definitive,altered gene product structure,NaN,undetermined,inferred,NaN,HP:0000006; HP:0008069; HP:0001036; HP:0003220...,30942823,Skin disorders,NaN,2025-01-09 15:40:24+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03623


<div class="alert alert-block alert-info">    
    
__Review of OMIM vs MONDO NodeNorm mismatches (22)__

None were orphanet.    
    
---

__19: OMIM's mapping is better__

> __6: Mondo ID is related but wrong__ -> __TELL EBI GENE2PHENO__
> * 'OMIM:243310', 'MONDO:0013812': omim is correct syndrome 1, but mondo is syndrome 2 (diff gene)
> * 'OMIM:613575', 'MONDO:0044314': omim is correct 55, but mondo is 78 (diff gene)
> * 'OMIM:101000', 'MONDO:0008075': omim is correct type of schwannomatosis (NF2/type 2), vs mondo is a sibling. 
>   * NodeNorm should map omim to MONDO:0007039 but isn't -> messaged NodeNorm
> * 'OMIM:613987', __'MONDO:0009136'__: omim is correct recessive 2, but mondo is recessive 1 (diff gene? Confusing because Monarch page links to gene NHP2 but OMIM page doesn't)
>   * NodeNorm should map omim to MONDO:0013519 but isn't -> messaged NodeNorm  
> * 'OMIM:613988', 'MONDO:0009136': omim is correct recessive 3, but mondo is recessive 1 (diff gene)
>   * NodeNorm should map omim to MONDO:0013520 but isn't -> messaged NodeNorm
> * 'OMIM:616353', 'MONDO:0009136': omim is correct recessive 6, but mondo is recessive 1 (diff gene)
>   * NodeNorm should map omim to MONDO:0014600 but isn't -> messaged NodeNorm

> __13: Mondo ID is too general__ (can see on Monarch website) -> __TELL EBI GENE2PHENO?__
> * 'OMIM:300696', 'MONDO:0010680': omim maps to MONDO:0010401, child of the mondo
> * 'OMIM:304120', 'MONDO:0019027': omim maps to MONDO:0010571 (syndrome type 2), child of the mondo (syndrome)
> * 'OMIM:610019', 'MONDO:0005129': omim maps to MONDO:0012395 (cataract 18), child of the mondo (cataract)
> * 'OMIM:611726', 'MONDO:0016295': omim maps to MONDO:0012721, child of the mondo 
> * 'OMIM:602668', 'MONDO:0016107': omim maps to MONDO:0011266 (type 2), child of the mondo
> * 'OMIM:203200', 'MONDO:0018910': omim maps to MONDO:0008746 (type 2), child of the mondo
> * 'OMIM:614328', 'MONDO:0017411': omim maps to MONDO:0013693 (type 1), child of the mondo
> * 'OMIM:175800', 'MONDO:0006602': omim maps to MONDO:0008290 (1, mibelli type), grandchild of the mondo
> * 'OMIM:614073', **'MONDO:0019312'**: omim maps to MONDO:0013556 (syndrome 4), child of the mondo (syndrome)
> * 'OMIM:614074', 'MONDO:0019312': omim maps to MONDO:0013557 (syndrome 5), child of the mondo (syndrome)
> * 'OMIM:614075', 'MONDO:0019312': omim maps to MONDO:0013558 (syndrome 6), child of the mondo (syndrome)
> * 'OMIM:614076', 'MONDO:0019312': omim maps to MONDO:0013559 (syndrome 7), child of the mondo (syndrome)
> * 'OMIM:614077', 'MONDO:0019312': omim maps to MONDO:0013560 (syndrome 8), child of the mondo (syndrome)

    
**1: MONDO's mapping is better**
<br>
Omim ID is slightly off -> __TELL EBI GENE2PHENO?__
* 'OMIM:613723', 'MONDO:0009181': mondo matches the disease name and phenotypes listed in the record better than the omim 


**1: Unsure**
* 'OMIM:158350', 'MONDO:0017623': omim is for Cowden syndrome 1, mondo is for PTEN hamartoma tumor syndrome. These are very similar, so I'm not sure which one is better. -> __TELL EBI GENE2PHENO?__
  * There's also another record w/ just the OMIM ID. I think the two rows should be merged. -> __TELL EBI GENE2PHENO?__


**1: NodeNorm error** -> messaged NodeNorm
* 'OMIM:224230', 'MONDO:0009136': both are recessive 1, NodeNorm should map to same entity

**Other rows reviewed:**
* 'OMIM:614583', 'MONDO:0013812': map to same correct entity

The prelim decision is to use disease OMIM/orphanet IDs because:
* less missing values
* more accurate in cases where there's also a MONDO ID

#### Checking MONDO data

Above, I decided the OMIM/orphanet disease IDs were better. 

However, I wondered if the MONDO IDs were accurate to the disease name when there weren't OMIM/orphanet IDs. Then they could be used for NodeNorming and less data would be dropped because it wasn't pre-NodeNormed. 

In [56]:
## get the data that has MONDO, doesn't have OMIM/orphanet

df_mondo_only = df[df["disease_mim"].isna() & df["disease_MONDO"].notna()].copy()

mondo_only = df_mondo_only["disease_MONDO"].dropna().unique()

In [57]:
## saving stats on data with only MONDO ID

stats_mondo_only = {
    "n_rows": df_mondo_only.shape[0],
    "n_names": len(mondo_only)
}

stats_mondo_only["n_rows"]
stats_mondo_only["n_names"]

444

277

In [58]:
## code chunk used to review some of the data

# df_mondo_only[df_mondo_only["disease_MONDO"] == mondo_only[240]]

df_mondo_only[df_mondo_only["panel"].str.contains("Skeletal", na=False)]

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,g2p_record_url
413,G2P03497,POP1,OMIM:602486,HGNC:30129,NaN,POP1-related anauxetic dysplasia,NaN,MONDO:0011773,biallelic_autosomal,NaN,definitive,decreased gene product level,NaN,undetermined,inferred,NaN,NaN,21455487; 27380734; 28067412,Skeletal disorders,NaN,2023-12-20 09:04:04+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03497


<div class="alert alert-block alert-info">    
    
__Reviewed some of the data__

Method: look at individual MONDO IDs. Covered all panels (only 1 skeletal, no hearing). 3 from earlier review (related to mismatches) + idx 0-240, step 10 + skeletal. 
    
__Summary__
* 37 rows (29 unique MONDO)
* __~16%__ were wrong (6/37) 
* Could tell EBI gene2pheno of issues but they are similar to those listed in mismatch mapping section

__Details__

__6 MONDO is related but wrong__
* "MONDO:0009136" for "RTEL1-related dyskeratosis congenita" (two rows): mondo is recessive 1, which is wrong. Should be recessive 5 MONDO:0014076/OMIM:615190 (old/synonym name is dominant 4) 
* "MONDO:0044314" for 4 rows "CLN3-related retinal dystrophy", "GUCA1B-", "PRPS1-", "SNRNP200-": mondo is type 78 (specifically for ARHGEF18), which is wrong. Should instead be:
  * CLN3 and PRPS1: a more general term like MONDO:0004580 (retinal degeneration) -> MONDO:0019118 (inherited retinal dystrophy) -> MONDO:0019200 (retinitis pigmentosa)
  * GUCA1B: type 48, MONDO:0013447
  * SNRNP200: type 33, MONDO:0012477
* "MONDO:0013522" for "TERC-related dyskeratosis congenita": mondo is for type 3 (specifically for TINF2, that row is in "Great" section). Should be type 1 MONDO:0007485/OMIM:127550. (confusing because Monarch's page of type 1 includes TINF2 and TERT too, but OMIM page only includes TERC)


__4 MONDO is too general__ 
* "MONDO:0020341" (periventricular nodular heterotopia) for "ERMARD-related periventricular heterotopia". The ERMARD-specific version is a child term: MONDO:0014240/OMIM:615544 (type 6)
* "MONDO:0018965" (Alport syndrome) for "COL4A5-related Alport syndrome". The COL4A5-specific version is a child term: MONDO:0010520/OMIM:301050  (X-linked)
* "MONDO:0024676" for "REST-related Wilms tumour": The REST-specific version is a **related** term: MONDO:0014779/OMIM:616806 (type 6)
* "MONDO:0011773" for "POP1-related anauxetic dysplasia": the POP1-specific version is a child term: MONDO:0054561/OMIM:617396 (type 2)


__4 Unsure__
* "MONDO:0005129" for "CYP51A1-related congenital cataract": mondo is cataract, which is not wrong but kinda general. MONDO:0033853 seems better (correlated with gene, matches phenotypes, orphanet ref uses one of the ref papers) -> TELL EBI GENE2PHENO?
* "MONDO:0018869" for "TMTC3-related cobblestone lissencephaly": while the mondo (cobblestone lissencephaly) sounds correct, it isn't linked to this gene. VS another sibling disease is linked to the gene, matches phenotypes, uses same paper: MONDO:0014992/OMIM:617255 (lissencephaly 8)
* "MONDO:0100100" for "SELENON-related myopathy": while mondo has exact name match, it's not directly linked to gene. Instead, its child disease is directly linked to gene MONDO:0011271/OMIM:602771 (rigid spine muscular dystrophy 1)
* "MONDO:0020367" for "MYOC-related juvenile open angle glaucoma": while mondo is almost-exact name match, it's not directly linked to gene. Instead, its child disease is directly linked to gene MONDO:0007664/OMIM:137750 (glaucoma 1, open angle, A) 


__5 Okay (using general term is fine)__
* "MONDO:0005129" for 3 other rows "WDR87-related congenital cataract", "AKR1E2-", "MFSD6L-": couldn't find better mappings. 
* "MONDO:0015469" for "DHRS3 related craniosynostosis": couldn't find better mapping
* "MONDO:0024676" (childhood kidney Wilms tumor) for "CTR9-related Wilms tumour", "TRIM28-": couldn't find better mapping. TRIM28 is correlated to parent term (kidney Wilms tumor). 


__18 Great__
* "MONDO:0012506" for "DSC2-related arrhythmogenic right ventricular cardiomyopathy"
* "MONDO:0011001" for "SCN5A-related Brugada syndrome"
* "MONDO:0013262" for "MYH7-related dilated cardiomyopathy"
* "MONDO:0013369" for "TNNI3-related hypertrophic cardiomyopathy"
* "MONDO:0010946" for "PRKAG2-related cardiomyopathy"
* "MONDO:0014143" for "RIT1-related Noonan syndrome"
* "MONDO:0010015" for "PXDN-related anterior segment dysgenesis with sclerocornea"
* "MONDO:0014214" for "DYNC2I1-related short-rib polydactyly"
* "MONDO:0013522" for "TINF2-related dyskeratosis congenita"
* "MONDO:0032876" for "WASF1-related intellectual disability with seizures"
* "MONDO:0859164" for "UNC45A-related osteootohepatoenteric syndrome"
* "MONDO:0018772" for "SLC30A7-related Joubert syndrome": using general term is fine since there isn't any established subtype of Joubert syndrome for this gene
* "MONDO:0010215" for "ERCC4-related xeroderma pigmentosum, group F"
* "MONDO:0009735" for "SPINK5-related Netherton syndrome"
* "MONDO:0007808" for "KRT1-related ichthyosis hystrix, Curth-Macklin type"
* "MONDO:0007566" for "TGFBR1-related multiple self-healing squamous epithelioma"
* "MONDO:0008285" for "PDGFRA-related gastrointestinal stromal tumor/GIST-plus syndrome, somatic or familial"
* "MONDO:0010912" for "TUBB3-related fibrosis of extraocular muscles, congenital"

#### Conclusions

<div class="alert alert-block alert-success">

__Exploration__

* some rows have no disease IDs
* a few NodeNorm mapping failures for OMIM IDs (several diff kinds): ~2.8%. 68 failures / (2401 unique values in column - 9 orphanet)
  * no NodeNorm mapping failures for MONDO IDs
* when rows have both OMIM and MONDO IDs, there are sometimes differences in NodeNorm mapping ("mismatches"). __In these cases, OMIM IDs were much more accurate__
* __MONDO IDs can be inaccurate__ - see the blue review boxes
  * VS it was much rarer to find an inaccurate OMIM ID mapping (found 1 case)


__Decision: Use OMIM ID column to generate NodeNorm values__

* less missing values
* seems to be more accurate (for successful NodeNorm mappings)

## Stats on rows removed

This section prints the statistics on rows in the original data that were removed. 

(Uses variables generated during the previous section "Pre-NodeNorming")

<div class="alert alert-block alert-success">

Genes: No rows removed due to lack of IDs for NodeNorming or NodeNorm mapping issues.

In [59]:
## partial put into parser (format): DONE

print("Gene Pre-NodeNorming\n")

## no gene IDs
print(f'{stats_no_gene_IDs["n_rows"]} row(s) with no gene IDs')

## no HGNC IDs: key column for NodeNorming
print(f'{n_rows_no_hgnc} row(s) with no HGNC IDs')

## HGNC NodeNorm issues: none, but showing anyways
print("\n")
print("HGNC NodeNorm mapping failures:")

print(f'IDs with no data in NodeNorm: {len(stats_hgnc_mapping_failures["nodenorm_returned_none"])}')
print(f'IDs with the wrong NodeNormed category: {len(stats_hgnc_mapping_failures["wrong_category"])}')
print(f'IDs with no label in NodeNorm: {len(stats_hgnc_mapping_failures["no_label"])}')

Gene Pre-NodeNorming

0 row(s) with no gene IDs
0 row(s) with no HGNC IDs


HGNC NodeNorm mapping failures:
IDs with no data in NodeNorm: 0
IDs with the wrong NodeNormed category: 0
IDs with no label in NodeNorm: 0


<div class="alert alert-block alert-success">

__Diseases: many rows removed__ due to lack of IDs for NodeNorming or NodeNorm mapping issues.

In [60]:
## partial put into parser (format): DONE

print("Disease Pre-NodeNorming\n")

## no disease IDs
print(f'{stats_no_disease_IDs["n_rows"]} row(s) with no disease IDs '
      f'(= {stats_no_disease_IDs["n_names"]} unique diseases)')

## plus the rows that only lack OMIM IDs: key column for NodeNorming
print(f'+ {stats_mondo_only["n_rows"]} row(s) with no OMIM ID '
      f'(= {stats_mondo_only["n_names"]} unique diseases)')

## OMIM NodeNorm issues
print("\n")
print("OMIM NodeNorm mapping failures:")

print(f'{n_rows_no_data} row(s) for '
      f'{len(stats_OmOr_mapping_failures["nodenorm_returned_none"])} '
      f'IDs with no data in NodeNorm')

print(f'{n_rows_wrong_category} row(s) for '
      f'{len(stats_OmOr_mapping_failures["wrong_category"])} '
      f'IDs with the wrong NodeNormed category')

print(f'{n_rows_no_label} row(s) for '
      f'{len(stats_OmOr_mapping_failures["no_label"])} '
      f'IDs with no label in NodeNorm')

Disease Pre-NodeNorming

633 row(s) with no disease IDs (= 621 unique diseases)
+ 444 row(s) with no OMIM ID (= 277 unique diseases)


OMIM NodeNorm mapping failures:
41 row(s) for 39 IDs with no data in NodeNorm
27 row(s) for 26 IDs with the wrong NodeNormed category
3 row(s) for 3 IDs with no label in NodeNorm


<div class="alert alert-block alert-success">
    
__Totals__

In [61]:
## put into parser (format): DONE

n_rows_original = df.shape[0]
n_rows_removed = stats_no_disease_IDs["n_rows"] + stats_mondo_only["n_rows"] + \
                 n_rows_no_data + n_rows_wrong_category + n_rows_no_label
n_rows_after = n_rows_original - n_rows_removed

print(f"{n_rows_original} rows/records before Pre-NodeNorming\n")

print(f"{n_rows_removed} rows removed during Disease NodeNorming process\n")

print(f"{n_rows_after} rows/records left ({n_rows_after/n_rows_original:.1%})")

3647 rows/records before Pre-NodeNorming

1148 rows removed during Disease NodeNorming process

2499 rows/records left (68.5%)


## Adding NodeNorm data, removing rows

Using gene HGNC and disease OMIM/orphanet IDs for pre-NodeNorming

In [62]:
## put into parser (format): DONE

## Gene: assumes no missing values
df["gene_nodenorm_id"] = [hgnc_nodenorm_mapping[i]["primary_id"] for i in df["hgnc_id"]]
df["gene_nodenorm_label"] = [hgnc_nodenorm_mapping[i]["primary_label"] for i in df["hgnc_id"]]

df["disease_nodenorm_id"] = [OmOr_nodenorm_mapping[i]["primary_id"] 
                             if OmOr_nodenorm_mapping.get(i) 
                             else pd.NA
                             for i in df["disease_mim"]]

df["disease_nodenorm_label"] = [OmOr_nodenorm_mapping[i]["primary_label"] 
                                if OmOr_nodenorm_mapping.get(i) 
                                else pd.NA
                                for i in df["disease_mim"]]

In [63]:
df.head()

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,g2p_record_url,gene_nodenorm_id,gene_nodenorm_label,disease_nodenorm_id,disease_nodenorm_label
0,G2P00124,KCNE1,OMIM:176261,HGNC:6240,ISK; JLNS2; LQT5; MINK,KCNE1-related Jervell and Lange-Nielsen syndrome,OMIM:612347,NaN,biallelic_autosomal,potential secondary finding,strong,altered gene product structure,missense_variant; inframe_deletion; stop_gaine...,undetermined,inferred,NaN,HP:0000407; HP:0001657; HP:0000007; HP:0001279,30461122,Developmental disorders; Cardiac disorders,KCNE1-related JLNS is due to altered gene prod...,2024-04-05 12:05:01+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00124,NCBIGene:3753,KCNE1,MONDO:0012871,Jervell and Lange-Nielsen syndrome 2
1,G2P00841,PTPN11,OMIM:176876,HGNC:9644,BPTP3; NS1; PTP2C; SH-PTP2; SHP-2; SHP2,PTPN11-related Noonan syndrome with multiple l...,OMIM:151100,NaN,monoallelic_autosomal,NaN,definitive,altered gene product structure,missense_variant; inframe_deletion; inframe_in...,undetermined,inferred,NaN,HP:0000325; HP:0002996; HP:0000957; HP:0001709...,27484170; 26377839; 25917897; 25884655; 248207...,Developmental disorders; Skin disorders; Cardi...,Expert review done on 12/01/2022; Noonan syndr...,2025-01-21 14:56:43+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00841,NCBIGene:5781,PTPN11,MONDO:0100082,LEOPARD syndrome 1
2,G2P03247,DSC2,OMIM:125645,HGNC:3036,CDHF2; DSC3,DSC2-related arrhythmogenic right ventricular ...,NaN,MONDO:0012506,monoallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,inframe_deletion; splice_region_variant; misse...,undetermined,inferred,NaN,NaN,31028357; 23911551; 21636032; 33831308; 263105...,Cardiac disorders,Expert review done on 05/01/2022; DSC2-related...,2024-03-20 09:36:09+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03247,NCBIGene:1824,DSC2,<NA>,<NA>
3,G2P03248,DSC2,OMIM:125645,HGNC:3036,CDHF2; DSC3,DSC2-related arrhythmogenic right ventricular ...,NaN,MONDO:0012506,biallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,inframe_deletion; splice_region_variant; misse...,undetermined,inferred,NaN,NaN,31028357; 23911551; 21636032; 33831308; 263105...,Cardiac disorders,Expert review done on 05/01/2022; DSC2-related...,2024-03-20 09:35:19+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03248,NCBIGene:1824,DSC2,<NA>,<NA>
4,G2P03249,DSG2,OMIM:125671,HGNC:3049,CDHF5,DSG2-related arrhythmogenic right ventricular ...,NaN,MONDO:0012434,monoallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,inframe_deletion; missense_variant; stop_gaine...,undetermined,inferred,NaN,NaN,21636032; 33831308; 33917638; 34400560; 240707...,Cardiac disorders,Expert review done on 05/01/2022; DSG2-related...,2024-03-20 09:40:18+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03249,NCBIGene:1829,DSG2,<NA>,<NA>


In [64]:
## put into parser (change in-place): DONE

df_only_nodenormed = df.dropna(subset=["gene_nodenorm_id", "gene_nodenorm_label", 
                                       "disease_nodenorm_id", "disease_nodenorm_label"],
                              ignore_index=True).copy()

In [65]:
## same! so it works as expected

df_only_nodenormed.shape

n_rows_after

(2499, 26)

2499

In [66]:
df_only_nodenormed.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2499 entries, 0 to 2498
Data columns (total 26 columns):
 #   Column                              Non-Null Count  Dtype              
---  ------                              --------------  -----              
 0   g2p_id                              2499 non-null   object             
 1   gene_symbol                         2499 non-null   object             
 2   gene_mim                            2499 non-null   object             
 3   hgnc_id                             2499 non-null   object             
 4   previous_gene_symbols               2247 non-null   object             
 5   disease_name                        2499 non-null   object             
 6   disease_mim                         2499 non-null   object             
 7   disease_MONDO                       115 non-null    object             
 8   allelic_requirement                 2499 non-null   object             
 9   cross_cutting_modifier              284 n

## Generating documents

In [67]:
## code chunk for testing parts of inner code

for row in df.itertuples(index=False):
    if pd.notna(row.previous_gene_symbols):
        [i.strip() for i in row.previous_gene_symbols.split(";")]
        break

['ISK', 'JLNS2', 'LQT5', 'MINK']

In [68]:
## put into parser (format): DONE
##   don't save in array, yield each document instead

## GENERATING DOCS, saving in array
documents = []

## using itertuples because it's faster, preserves datatypes
for row in df_only_nodenormed.itertuples(index=False):
    ## simple assignments
    document = {
        "_id": row.g2p_id,
        "subject": {
            "id": row.gene_nodenorm_id,
            "label": row.gene_nodenorm_label,
            "original_subject": row.hgnc_id,
            "original_data": {
                "hgnc_symbol": row.gene_symbol,
                "hgnc": row.hgnc_id,
            },
            "type": "Gene"
        },
        "association": {
            "g2p_record_id": row.g2p_id,
            "g2p_record_url": row.g2p_record_url,
            "allelic_requirement": row.allelic_requirement,
            "confidence": row.confidence,
            "molecular_mechanism": row.molecular_mechanism,
            "molecular_mechanism_categorisation": row.molecular_mechanism_categorisation,
            "g2p_panels": [i.strip() for i in row.panel.split(";")],
            "date_of_last_review": str(row.date_of_last_review)
        },
        "object": {
            "id": row.disease_nodenorm_id,
            "label": row.disease_nodenorm_label,
            ## currently, after NodeNorming, no NAs in OMIM/orphanet column
            "original_object": row.disease_mim,
            "original_data": {
                ## not putting disease_mim here: parsing into separate OMIM and orphanet fields 
                "name": row.disease_name,
            },
            "type": "Disease"
        }
    }    
    ## only create field if value is not NA
    ##   if value is NA, list comprehension with split won't work
    ## Gene
    if pd.notna(row.gene_mim):
        document["subject"]["original_data"]["omim"] = row.gene_mim
    if pd.notna(row.previous_gene_symbols):
        document["subject"]["original_data"]["previous_gene_symbols"] = \
        [i.strip() for i in row.previous_gene_symbols.split(";")]
        
    ## Association
    if pd.notna(row.cross_cutting_modifier):
        document["association"]["cross_cutting_modifiers"] = [i.strip() for i in row.cross_cutting_modifier.split(";")]
    if pd.notna(row.variant_consequence):
        document["association"]["variant_consequences"] = [i.strip() for i in row.variant_consequence.split(";")]
    if pd.notna(row.variant_types):
        document["association"]["variant_types"] = [i.strip() for i in row.variant_types.split(";")]
    ## uses diff delimiters, could do more parsing
    if pd.notna(row.molecular_mechanism_evidence):
        document["association"]["molecular_mechanism_evidence"] = [i.strip() for i in row.molecular_mechanism_evidence.split("&")]
    if pd.notna(row.phenotypes):
        document["association"]["phenotypes"] = [i.strip() for i in row.phenotypes.split(";")]
    if pd.notna(row.publications):
        document["association"]["pmids"] = [i.strip() for i in row.publications.split(";")]
    if pd.notna(row.comments):
        document["association"]["curator_comments"] = row.comments
     
    ## Disease
    ## disease_mim: create field depending on whether OMIM or orphanet    
    if pd.notna(row.disease_mim):
        if row.disease_mim.startswith("orphanet"):
            document["object"]["original_data"]["orphanet"] = row.disease_mim
        elif row.disease_mim.startswith("OMIM"):
            document["object"]["original_data"]["omim"] = row.disease_mim
    if pd.notna(row.disease_MONDO):
        document["object"]["original_data"]["mondo"] = row.disease_MONDO
    
    documents.append(document)

## Checking documents

In [69]:
len(documents)

2499

In [70]:
## code chunk for finding rows -> then look up corresponding doc by idx
# df_only_nodenormed[df_only_nodenormed["disease_mim"].str.contains("orphanet", na=False)]


# df_only_nodenormed[df_only_nodenormed["previous_gene_symbols"].isna()]
# df_only_nodenormed[df_only_nodenormed["disease_MONDO"].notna()]

In [71]:
pprint(documents[416])

# documents[416]

{'_id': 'G2P00116',
 'association': {'allelic_requirement': 'biallelic_autosomal',
                 'confidence': 'definitive',
                 'date_of_last_review': '2015-07-22 16:14:16+00:00',
                 'g2p_panels': ['Developmental disorders'],
                 'g2p_record_id': 'G2P00116',
                 'g2p_record_url': 'https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00116',
                 'molecular_mechanism': 'loss of function',
                 'molecular_mechanism_categorisation': 'inferred',
                 'phenotypes': ['HP:0003281',
                                'HP:0001903',
                                'HP:0003593',
                                'HP:0001425',
                                'HP:0000007'],
                 'pmids': ['19412178'],
                 'variant_consequences': ['absent gene product']},
 'object': {'id': 'MONDO:0008785',
            'label': 'sideroblastic anemia 2',
            'original_data': {'name': 'SLC25A38-related anemia

## Parser notes

Fine to use raise/assert in parser (raise is technically better programming behavior: https://realpython.com/python-assert-statement/#understanding-common-pitfalls-of-assert)


My notes on parser:
* adding prefixes to gene/disease IDs is good for pre-NodeNorming steps
* keeping diff gene/disease ID namespaces as separate fields right now is good for current BTE/x-bte-annotation system
  * Also, original subject will always be HGNC, original object will always be disease OMIM with current code


My notes on syntax:
* use `yield` when you want to "return" within a "for loop" (return only happen once, then exit for-loop/function execution)
  * that's what it's used in main execution, when you're iterating over csv rows to generate documents
* use `yield from {function}` to get the data from a generator (created by `yield` being used the function)